In [2]:
import sys
import csv
import sklearn
import unicodedata
import math
from nltk.tokenize import *

In [14]:
INPUT_FILE = sys.argv[1]
OUTPUT_FILE = sys.argv[2]
# 6 categories and 21 subcategories
word_counts = {"1":{},"2":{},"3":{},"4":{},"5":{},"6":{}}
vocab = {}
category_prior = {"1":0.0,"2":0.0,"3":0.0,"4":0.0,"5":0.0,"6":0.0}
subcategory_prior = {}
row_num = 1

In [17]:

for i in xrange(22):
    subcategory_prior[i] = 0.0
print subcategory_prior
print category_prior


{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0}
{'1': 0.0, '3': 0.0, '2': 0.0, '5': 0.0, '4': 0.0, '6': 0.0}


In [5]:
def count_words(words):
    wc = {}
    for word in words:
        wc[word.encode("ascii")] = wc.get(word.encode("ascii"),0.0) + 1.0
    
    return wc

<h2> for training data </h2>

In [29]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
with open('feature_svm_train.csv','rb') as readfile:
    with open(OUTPUT_FILE,'w') as writefile:
        
        reader = csv.reader(readfile, skipinitialspace =False, quoting=csv.QUOTE_MINIMAL)
        writer = csv.writer(writefile, quoting = csv.QUOTE_MINIMAL)
        
        for row in reader:
            if row_num<=20000:
                x = tknzr.tokenize(row[0])
                for words in x:
                    words.encode("ascii","ignore")
                category_prior[row[1]] += 1
                subcategory_prior[int(row[2])] +=1
                counts = count_words(x)
                
                #print counts
                for word, count in list(counts.items()):
                    if word not in vocab:
                        vocab[word] = 0.0
                    if word not in word_counts[row[1]]:
                        word_counts[row[1]][word] = 0.0
                    vocab[word] = vocab[word] + count
                    word_counts[row[1]][word]+=count
                row_num += 1
            else:
                break
        print category_prior
        
                

{'1': 4866.0, '3': 11.0, '2': 3921.0, '5': 4102.0, '4': 1389.0, '6': 5712.0}


<h2> For samples </h2>

In [41]:
with open('feature_svm_test.csv') as examplefile:
    reader = csv.reader(examplefile, skipinitialspace =False, quoting=csv.QUOTE_MINIMAL)
    list_item = 1
    correct = 0
    prob_word_given_label =[]
    prob_label_given_word = []
    for row in reader:
        x = tknzr.tokenize(row[0])
        for words in x:
            words.encode("ascii","ignore")
            
        counts = count_words(x)
        
        category_classification = {"1":0.0,"2":0.0,"3":0.0,"4":0.0,"5":0.0,"6":0.0}
        for word, count in list(counts.items()):
            if word not in vocab:
                continue
            for i in range(1,len(list(word_counts.values()))+1):
                prob_word_given_label.append(word_counts[str(i)].get(word,0.0)/sum(word_counts[str(i)].values()))
            print prob_word_given_label
            for i in range(1,len(prob_word_given_label)):
                prob_label_given_word.append(prob_word_given_label[i] * category_prior[str(i)]/sum(category_prior.values()))
            print prob_label_given_word
            

[0.0020405904236631485, 0.0022410653995614542, 0.0, 0.0012316663943594706, 0.0013035182856671245, 0.0006741740765620974]
[0.0005452239505157761, 0.0, 6.773826477653205e-07, 9.052481869864687e-05]
[0.0020405904236631485, 0.0022410653995614542, 0.0, 0.0012316663943594706, 0.0013035182856671245, 0.0006741740765620974, 9.866062189880789e-05, 3.808346430627308e-05, 0.0, 0.00023879246421255044, 8.495929236191687e-05, 3.9728115225980744e-05]


KeyError: '7'